In [1]:
## DatabaseUtils.py
import sqlite3
import os
import json

class DatabaseUtils:
    def __init__(self, db_name="IntelliMateClient.db"):
        self.db_name = db_name
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()

    def create_database(self):
            # Create Users Table
            self.cursor.execute("""
                CREATE TABLE Users (
                    user_id TEXT PRIMARY KEY,
                    username TEXT,
                    password TEXT,
                    status TEXT,
                    last_login TEXT,
                    last_updated TEXT,
                    active_conversations TEXT,
                    usage_stats TEXT,
                    preferences TEXT,
                    tags TEXT,
                    version TEXT,
                    custom_settings TEXT
                );
            """)

            # Create Assistants Table
            self.cursor.execute("""
                CREATE TABLE Assistants (
                    assistant_id TEXT PRIMARY KEY,
                    assistant_name TEXT,
                    assistant_model TEXT,
                    persona TEXT,
                    likes TEXT,
                    dislikes TEXT,
                    relationship TEXT,
                    expertise TEXT,
                    status TEXT,
                    tags TEXT,
                    last_active TEXT,
                    active_conversations TEXT,
                    language TEXT,
                    version TEXT,
                    custom_settings TEXT
                );
            """)

            # Create Conversations Table
            self.cursor.execute("""
                CREATE TABLE Conversations (
                    conversation_id TEXT PRIMARY KEY,
                    messages TEXT,
                    info TEXT
                );
            """)

            # Commit changes and close connection
            self.conn.commit()
            self.conn.close()
            print("Database and tables created successfully.")

    def create_user(self, user_data):
        sql = """
            INSERT INTO Users (user_id, username, password, status, last_login, last_updated,
                               active_conversations, usage_stats, preferences, tags, version, custom_settings)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        self.cursor.execute(sql, (
            user_data['user_id'], user_data['username'], user_data['password'], user_data['status'],
            user_data['last_login'], user_data['last_updated'], json.dumps(user_data['active_conversations']),
            json.dumps(user_data['usage_stats']), json.dumps(user_data['preferences']), json.dumps(user_data['tags']),
            user_data['version'], json.dumps(user_data['custom_settings'])
        ))
        self.conn.commit()

    def read_user(self, user_id):
        sql = "SELECT * FROM Users WHERE user_id = ?"
        self.cursor.execute(sql, (user_id,))
        return self.cursor.fetchone()

    def update_user(self, user_id, updates):
        for field, value in updates:
            sql = f"UPDATE Users SET {field} = ? WHERE user_id = ?"
            self.cursor.execute(sql, (json.dumps(value) if isinstance(value, (list, dict)) else value, user_id))
        self.conn.commit()

    def delete_user(self, user_id):
        sql = "DELETE FROM Users WHERE user_id = ?"
        self.cursor.execute(sql, (user_id,))
        self.conn.commit()

    def close(self):
        self.conn.close()

    def create_conversation(self, conversation_data):
        sql = """
            INSERT INTO Conversations (conversation_id, messages, info)
            VALUES (?, ?, ?)
        """
        self.cursor.execute(sql, (
            conversation_data['conversation_id'],
            json.dumps(conversation_data['messages']),
            json.dumps(conversation_data['info'])
        ))
        self.conn.commit()

    def read_conversation(self, conversation_id):
        sql = "SELECT * FROM Conversations WHERE conversation_id = ?"
        self.cursor.execute(sql, (conversation_id,))
        return self.cursor.fetchone()

    def update_conversation(self, conversation_id, updates):
        for field, value in updates:
            sql = f"UPDATE Conversations SET {field} = ? WHERE conversation_id = ?"
            self.cursor.execute(sql, (json.dumps(value) if isinstance(value, (list, dict)) else value, conversation_id))
        self.conn.commit()

    def delete_conversation(self, conversation_id):
        sql = "DELETE FROM Conversations WHERE conversation_id = ?"
        self.cursor.execute(sql, (conversation_id,))
        self.conn.commit()

    def close(self):
        self.conn.close()
    def create_assistant(self, assistant_data):
        sql = """
            INSERT INTO Assistants (assistant_id, assistant_name, assistant_model, persona,
                                    likes, dislikes, relationship, expertise, status, tags,
                                    last_active, active_conversations, language, version, custom_settings)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        self.cursor.execute(sql, (
            assistant_data['assistant_id'], assistant_data['assistant_name'], assistant_data['assistant_model'],
            assistant_data['persona'], json.dumps(assistant_data['likes']), json.dumps(assistant_data['dislikes']),
            assistant_data['relationship'], json.dumps(assistant_data['expertise']), assistant_data['status'],
            json.dumps(assistant_data['tags']), assistant_data['last_active'], json.dumps(assistant_data['active_conversations']),
            assistant_data['language'], assistant_data['version'], json.dumps(assistant_data['custom_settings'])
        ))
        self.conn.commit()

    def read_assistant(self, assistant_id):
        sql = "SELECT * FROM Assistants WHERE assistant_id = ?"
        self.cursor.execute(sql, (assistant_id,))
        return self.cursor.fetchone()

    def update_assistant(self, assistant_id, updates):
        for field, value in updates:
            sql = f"UPDATE Assistants SET {field} = ? WHERE assistant_id = ?"
            self.cursor.execute(sql, (json.dumps(value) if isinstance(value, (list, dict)) else value, assistant_id))
        self.conn.commit()

    def delete_assistant(self, assistant_id):
        sql = "DELETE FROM Assistants WHERE assistant_id = ?"
        self.cursor.execute(sql, (assistant_id,))
        self.conn.commit()

    def close(self):
        self.conn.close()


In [2]:
## Conversation class
import json
import datetime

class Conversation:
    def __init__(self, conversation_id, start_time):
        self.data = {
            'messages': [],
            'info': {
                'conversation_id': conversation_id,
                'start_time': start_time,
                'end_time': None,
                'status': 'active'
            }
        }

    def add_message(self, role, content, model=None):
        message = {
            'role': role,
            'content': content,
            'timestamp': datetime.datetime.now().isoformat(),
            'model': model,
            'token_count': len(content.split()),
            'message_id': len(self.data['messages']) + 1,
            'response_time': None  # This can be updated later
        }
        self.data['messages'].append(message)

    def get_messages(self):
        return self.data['messages']

    def get_last_message(self):
        return self.data['messages'][-1] if self.data['messages'] else None

    def get_message_by_id(self, message_id):
        for message in self.data['messages']:
            if message['message_id'] == message_id:
                return message
        return None

    def update_message(self, message_id, new_content):
        message = self.get_message_by_id(message_id)
        if message:
            message['content'] = new_content
            message['token_count'] = len(new_content.split())

    def delete_message(self, message_id):
        self.data['messages'] = [msg for msg in self.data['messages'] if msg['message_id'] != message_id]

    def end_conversation(self, end_time):
        self.data['info']['end_time'] = end_time
        self.data['info']['status'] = 'inactive'

    def get_info(self):
        return self.data['info']

    def to_json(self):
        return json.dumps(self.data)

    def from_json(self, json_str):
        self.data = json.loads(json_str)


In [3]:
## User class
import json

class User:
    def __init__(self, user_id, username, password, language, version):
        self.data = {
            "user_id": user_id,
            "username": username,
            "password": password,  # In a real-world application, this should be hashed
            "status": "active",
            "last_login": None,
            "last_updated": None,
            "active_conversations": [],
            "usage_stats": {
                "total_tokens": 0,
                "total_queries": 0,
                "credit_balance": 0.0
            },
            "preferences": {
                "language": language,
                "notifications": "on"
            },
            "tags": [],
            "version": version,
            "custom_settings": {}
        }

    def login(self):
        # Update the last_login timestamp and set status to active
        # Placeholder for actual timestamp
        self.data["last_login"] = "current_timestamp"
        self.data["status"] = "active"

    def logout(self):
        # Update the status to inactive
        self.data["status"] = "inactive"

    def update_password(self, new_password):
        # Update the password
        self.data["password"] = new_password  # Again, this should be hashed in a real-world application

    def update_preferences(self, language, notifications):
        # Update user preferences
        self.data["preferences"]["language"] = language
        self.data["preferences"]["notifications"] = notifications

    def get_usage_stats(self):
        # Return the usage_stats dictionary
        return self.data["usage_stats"]

    def add_conversation(self, convo_id):
        # Add a conversation ID to the list of active conversations
        self.data["active_conversations"].append(convo_id)

    def remove_conversation(self, convo_id):
        # Remove a conversation ID from the list of active conversations
        self.data["active_conversations"].remove(convo_id)

    def to_json(self):
        # Serialize the data object to a JSON-formatted string
        return json.dumps(self.data)

    def from_json(self, json_str):
        # Deserialize a JSON-formatted string to populate the data object
        self.data = json.loads(json_str)


In [ ]:
## IntellimateClient class
class IntelliMateClient:
    def __init__(self, db_name="IntelliMateClient.db"):
        # Initialize dictionaries to hold User, Assistant, and Conversation objects
        self.users = {}
        self.assistants = {}
        self.conversations = {}
        
        # Initialize the DatabaseUtils instance for database operations
        self.db_utils = DatabaseUtils(db_name)
        
        # Optionally, load the initial state from the database
        self.load_state()

    def load_state(self):
        # Load users, assistants, and conversations from the database
        # This is a placeholder; the actual implementation will depend on how the data is stored
        pass


    # User Management Functions

    def create_user(self, user_id, username, password, language, version):
        # Create a new User object
        new_user = User(user_id, username, password, language, version)

        # Add the new user to the users dictionary
        self.users[user_id] = new_user

        # Convert the JSON string to a dictionary
        user_data_dict = json.loads(new_user.to_json())

        # Add the new user to the database
        self.db_utils.create_user(user_data_dict)

        return new_user

    def get_user(self, user_id):
        # Retrieve the user from the users dictionary
        return self.users.get(user_id, None)

    def update_user(self, user_id, update_fields):
        # Retrieve the user object
        user = self.get_user(user_id)
        
        if user:
            # Update the user object based on the list of tuples (field, new_value)
            for field, new_value in update_fields:
                setattr(user, field, new_value)
            
            # Update the user in the database
            self.db_utils.update_user(user.to_json())
            
            return True
        else:
            return False

    def delete_user(self, user_id):
        # Remove the user from the users dictionary
        if user_id in self.users:
            del self.users[user_id]
            
            # Remove the user from the database
            self.db_utils.delete_user(user_id)
            
            return True
        else:
            return False

# Example usage
if __name__ == "__main__":
    client = IntelliMateClient()
    new_user = client.create_user("1", "jakes", "password123", "en", "1.0")
    print("New User Created:", new_user.to_json())
    
    retrieved_user = client.get_user("1")
    print("Retrieved User:", retrieved_user.to_json() if retrieved_user else "User not found")
    
    client.update_user("1", [("username", "jakes_updated"), ("password", "new_password123")])
    updated_user = client.get_user("1")
    print("Updated User:", updated_user.to_json() if updated_user else "User not found")
    
    client.delete_user("1")
    deleted_user = client.get_user("1")
    print("Deleted User:", "User successfully deleted" if not deleted_user else "User still exists")

In [ ]:
## faker
import random
import json
from faker import Faker

# Assuming DatabaseUtils, User, Assistant, and Conversation classes are already defined or imported

fake = Faker()

# Initialize DatabaseUtils
db_utils = DatabaseUtils("IntelliMateClient.db")

# Generate fake users
for i in range(1, 11):
    user_id = str(i)
    username = fake.user_name()
    password = fake.password()
    language = random.choice(["en", "fr", "es", "de"])
    version = "1.0"
    
    new_user = User(user_id, username, password, language, version)
    user_data_dict = json.loads(new_user.to_json())  # Deserialize JSON string to dictionary
    db_utils.create_user(user_data_dict)

# Generate fake assistants
for i in range(1, 6):
    assistant_id = str(i)
    assistant_name = fake.first_name()
    assistant_model = "gpt-3.5-turbo"
    persona = "helpful, witty"
    likes = json.dumps(["python", "pandas", "music"])
    dislikes = json.dumps(["emacs", "politics"])
    relationship = "AI assistant/project partner"
    expertise = json.dumps(["data analytics", "research"])
    language = "en"
    version = "1.0"
    
    new_assistant = Assistant(assistant_id, assistant_name, assistant_model, language, version)
    assistant_data_dict = json.loads(new_assistant.to_json())  # Deserialize JSON string to dictionary
    db_utils.create_assistant(assistant_data_dict)

# Generate fake conversations
for i in range(1, 21):
    conversation_id = str(i)
    start_time = fake.date_time_this_year()
    new_conversation = Conversation(conversation_id, start_time)
    conversation_data_dict = json.loads(new_conversation.to_json())  # Deserialize JSON string to dictionary
    db_utils.create_conversation(conversation_data_dict)

print("Database populated with fake data!")